In [1]:
#include "stdio.h"
#include "stdlib.h"
#include <iostream>
#include <vector>

/*a workaround to solve cling issue*/
#include "../macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../load_libtorch.hpp"
#include <torch/torch.h>

# 1.创建一个Tensor

## 1.1 创建一个新的Tensor，以empty函数为例：

### 基础操作，使用官方API

从 [libtorch的c++ API文档](https://pytorch.org/cppdocs/index.html) 中可以得知，torch::empty()函数接口定义如下：        

![torch_empty function](./images/tensor_torch_empty.png)

主要是两个参数：IntArrayRef和TensorOptions。

In [9]:
torch::Tensor t1 = torch::empty(3, torch::kInt64);

std::cout << t1 << std::endl;

 8.3737e+18
 8.3917e+18
 5.0590e+18
[ CPULongType{3} ]


In [10]:
torch::Tensor t1 = torch::empty(3, torch::kCPU);

std::cout << t1 << std::endl;

-1.1741e-19
 4.5817e-41
-1.1741e-19
[ CPUFloatType{3} ]


In [12]:
torch::Tensor t2 = torch::empty({5,6}, torch::kInt32);
std::cout << t2 << std::endl;

 1.8699e+09  9.4637e+08  1.8699e+09  1.6841e+09  1.9672e+09  1.9194e+09
 1.6996e+09  1.3971e+09  1.3973e+09  1.9843e+09  1.3977e+09  1.5994e+09
 1.1639e+09  2.1760e+04  9.7000e+01  0.0000e+00 -1.1693e+09  2.1879e+04
-1.1714e+09  2.1879e+04  1.9516e+09  1.6676e+09  1.2322e+09  1.7150e+09
 1.9527e+09  1.2320e+09  1.6307e+09  1.7000e+09  1.9199e+09  1.3974e+09
[ CPUIntType{5,6} ]


### 深入一点了解，参数类型是怎么回事？

咋一看，上面例子中的参数貌似跟API函数中描述的并不一样，对于第一个参数，3显然是个整数，跟IntArrayRef如何关联？ 对于第二个参数，torch::kCPU是Device枚举类型，torch::kInt64是ScalarType枚举类型，好像与TensorOptions风牛马不相及，关于这些问题，我们就得看看相关参数类型的定义了：

#### IntArrayRef(c10/util/ArrayRef.h:274)

```
using IntArrayRef = ArrayRef<int64_t>;


/// Construct an ArrayRef from a std::array
template <size_t N>
/* implicit */ constexpr ArrayRef(const std::array<T, N>& Arr)
  : Data(Arr.data()), Length(N) {}

/// Construct an ArrayRef from a C array.
template <size_t N>
/* implicit */ constexpr ArrayRef(const T (&Arr)[N]) : Data(Arr), Length(N) {}
```

***可见IntArrayRef本质上还是一个int64的list，并且可以从std array或者c array进行构造***

---

#### TensorOptions(c10/core/TensorOptions.h)

```
/// Constructs a `TensorOptions` object with the given layout.
/* implicit */ TensorOptions(Layout layout) : TensorOptions() {
this->set_layout(layout);
}

/// Constructs a `TensorOptions` object with the given device.
/// See NOTE [ TensorOptions Constructors ] on why this is templatized.
template<typename T,
       typename = std::enable_if_t<std::is_same<std::decay_t<T>, Device>::value>>
/* implicit */ TensorOptions(T&& device) : TensorOptions() {
this->set_device(std::forward<T>(device));
}

/// legacy constructor to support ScalarType
/* implicit */ TensorOptions(ScalarType dtype) : TensorOptions() {
this->set_dtype(dtype);
}

```

***TensorOptions的隐式构造函数支持从Layout、Device和ScalarType三种枚举类型参数去进行构建，所以我们看到的上述例子也是OK的；***

---

*下面我们就根据上述API描述的参数来生成几个tensor：*

In [42]:
torch::Device dev_cpu(torch::kCPU);
/*因为只有一块1060，因此index只能填0，填其它值会报错*/
/*macbook pro就不要尝试了，因为你没有cuda！*/
torch::Device dev_gpu(torch::kCUDA, 0/*device index*/);

torch::ScalarType dtype_int(torch::kInt);

at::IntArrayRef m_iar_1({3}); 
at::IntArrayRef m_iar_2({3,4}); 
at::ArrayRef<int64_t> m_ar_1({3});
at::ArrayRef<int64_t> m_ar_2({3,4});

torch::Tensor t1 = torch::empty(m_iar_1, dev_cpu);
std::cout << "t1 = " << std::endl << t1 << std::endl << std::endl;

torch::Tensor t2 = torch::empty(m_ar_2, dev_gpu);
std::cout << "t2 = " << std::endl << t2 << std::endl << std::endl;

torch::Tensor t3 = torch::empty(m_ar_1, dtype_int);
std::cout << "t3 = " << std::endl << t3 << std::endl << std::endl;

torch::Tensor t4 = torch::empty(m_iar_2, torch::kSparse);
std::cout << "t4 = " << std::endl << t4 << std::endl << std::endl;


torch::Tensor t5 = torch::empty(m_iar_2, at::device(at::kCUDA).dtype(torch::kByte));
std::cout << "t5 = " << std::endl << t5 << std::endl << std::endl;


t1 = 
 4.5840e+30
 6.0621e+22
 3.1581e+03
[ CPUFloatType{3} ]

t2 = 
 0  0  0  0
 0  0  0  0
 0  0  0  0
[ CUDAFloatType{3,4} ]

t3 = 
 9.6267e+08
 1.7179e+09
 5.7386e+08
[ CPUIntType{3} ]

t4 = 
[ SparseCPUFloatType{}
indices:
[ CPULongType{2,0} ]
values:
[ CPUFloatType{0} ]
size:
[3, 4]
]

t5 = 
 0  0  0  0
 0  0  0  0
 0  0  0  0
[ CUDAByteType{3,4} ]



## 1.2 创建值为0的tensor，使用zero函数：

同torch::empty(...)函数一样，torch::zeros(...)函数的定义也是位于`{pytorch}/torch/csrc/autograd/generated/variable_factories.h`文件中，具体调用的，还是at::zeros(...)函数，其具体实现位于`{pytorch}/aten/src/ATen/native/TensorFactoryies.cpp`，大致的调用方式是torch::zeros(...) -> at::zeros(...) -> at::native::full(...) -> at::empty(...).fill_(...)


In [5]:
torch::Tensor x = torch::zeros({3, 4});
std::cout << x << std::endl;

torch::Tensor y = torch::zeros({3, 4}, torch::kInt);
std::cout << y << std::endl;

 0  0  0  0
 0  0  0  0
 0  0  0  0
[ CPUFloatType{3,4} ]
 0  0  0  0
 0  0  0  0
 0  0  0  0
[ CPUIntType{3,4} ]


创建值为随机数的tensor：

In [6]:
torch::Tensor x = torch::rand({3, 4});
std::cout << x << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

torch::Tensor y = torch::rand({3, 4}, torch::kFloat64);
std::cout << y << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

torch::Tensor z = torch::randn({5, 3, 4}, torch::kFloat32);
std::cout << z << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

 0.4444  0.7839  0.3972  0.5826
 0.8967  0.1082  0.5972  0.1979
 0.5112  0.3519  0.5890  0.8016
[ CPUFloatType{3,4} ]
<<<=========>>>

 0.3386  0.7420  0.4815  0.3888
 0.8127  0.1300  0.6292  0.4419
 0.7200  0.4255  0.8997  0.3681
[ CPUDoubleType{3,4} ]
<<<=========>>>

(1,.,.) = 
 -0.2787  1.2748  0.8307  0.9763
 -0.3594 -1.1021 -0.6364 -0.0364
  0.7180  0.0749 -0.0436 -2.4029

(2,.,.) = 
 -0.2440 -0.0459  0.8772 -0.9237
 -1.6146 -0.5144 -0.0731  0.2669
 -0.5208 -0.0864  0.9103  0.8287

(3,.,.) = 
 -0.8943 -1.7940  0.0125  1.6025
 -2.0548  0.3773  1.4870  0.5982
 -0.0344  0.8939  1.7970  0.5809

(4,.,.) = 
  1.1613 -1.6727 -1.1365  1.2555
 -0.1700  0.2928  1.7869 -0.2510
  0.2229  2.3172 -2.0089 -0.7028

(5,.,.) = 
 -0.1174 -0.4590 -0.6379  0.7281
  0.0454  0.5225 -0.3091  1.8815
  0.1755 -1.0258 -0.2639  0.5765
[ CPUFloatType{5,3,4} ]
<<<=========>>>



根据已有tensor创建新的tensor：

In [7]:
torch::Tensor t = torch::ones({3,3});
t += 2;


torch::Tensor a(t);
std::cout << a << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

 3  3  3
 3  3  3
 3  3  3
[ CPUFloatType{3,3} ]
<<<=========>>>



In [8]:
torch::Tensor a = torch::ones({5,3});
a = torch::randn_like(a);

std::cout << a << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

 0.1059  1.7008 -1.0940
 0.6755 -0.5386 -1.5082
 0.5866  0.4006  1.7089
-0.3359  0.1243 -0.2685
 1.2250 -1.8495  0.8217
[ CPUFloatType{5,3} ]
<<<=========>>>



更多例子请参考 [Tensor Creation API](https://pytorch.org/cppdocs/notes/tensor_creation.html)

# 2. Tensor的相关运算

## 加法运算

In [10]:
torch::Tensor x = torch::ones({5,3});
torch::Tensor y = torch::randn({5,3});

std::cout << "x =" << std::endl;
std::cout << (x) << std::endl << std::endl;

std::cout << "y =" << std::endl;
std::cout << (y) << std::endl << std::endl;

std::cout << "x+y =" << std::endl;
std::cout << (x+y) << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

std::cout << "torch::add(x,y) =" << std::endl;
std::cout << torch::add(x,y) << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

std::cout << "y.add_(x) =" << std::endl;
std::cout << y.add_(x) << std::endl;
std::cout << "<<<=========>>>" << std::endl << std::endl;

x =
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
[ CPUFloatType{5,3} ]

y =
-0.4262  0.2588 -1.2324
 1.5842 -1.4629  0.5925
-0.2029  0.6585 -0.0634
 2.2986 -0.6072  0.9584
 2.4718  1.3781  1.0643
[ CPUFloatType{5,3} ]

x+y =
 0.5738  1.2588 -0.2324
 2.5842 -0.4629  1.5925
 0.7971  1.6585  0.9366
 3.2986  0.3928  1.9584
 3.4718  2.3781  2.0643
[ CPUFloatType{5,3} ]
<<<=========>>>

torch::add(x,y) =
 0.5738  1.2588 -0.2324
 2.5842 -0.4629  1.5925
 0.7971  1.6585  0.9366
 3.2986  0.3928  1.9584
 3.4718  2.3781  2.0643
[ CPUFloatType{5,3} ]
<<<=========>>>

y.add_(x) =
 0.5738  1.2588 -0.2324
 2.5842 -0.4629  1.5925
 0.7971  1.6585  0.9366
 3.2986  0.3928  1.9584
 3.4718  2.3781  2.0643
[ CPUFloatType{5,3} ]
<<<=========>>>



## 索引
借助于torch::Tensor::index()和torch::Tensor::index_put_()函数，我们可以在libtorch中实现类似pytorch中对tensor的切片存取操作。具体说明详见[tensor_indexing](https://pytorch.org/cppdocs/notes/tensor_indexing.html)页面.

In [14]:
torch::Tensor x = torch::zeros(15);

for (int i = 0; i < 15; i++) {
    x[i] = i;
}
    
std::cout << "x =" << std::endl;
std::cout << (x) << std::endl << std::endl;

/* *
 *  Getter ops
 */
std::cout << "(in python) x[None]=" << std::endl;
std::cout << (x.index({torch::indexing::None})) << std::endl << std::endl;

std::cout << "(in python) x[Ellipsis, ...]=" << std::endl;
std::cout << (x.index({torch::indexing::Ellipsis, "..."})) << std::endl << std::endl;

std::cout << "(in python) x[7]=" << std::endl;
std::cout << (x.index({7})) << std::endl << std::endl;

std::cout << "(in python) x[True, False]=" << std::endl;
std::cout << (x.index({true,false,true,false,true,false,true,false,true,false,false,true,false,true,false})) << std::endl << std::endl;

std::cout << "(in python) x[1::2]=" << std::endl;
std::cout << (x.index({torch::indexing::Slice(1, torch::indexing::None, 2)})) << std::endl << std::endl;

std::cout << "(in python) x[::2]=" << std::endl;
std::cout << (x.index({torch::indexing::Slice(torch::indexing::None, torch::indexing::None, 2)})) << std::endl << std::endl;


x =
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
[ CPUFloatType{15} ]

(in python) x[None]=
  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14
[ CPUFloatType{1,15} ]

(in python) x[Ellipsis, ...]=
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
[ CPUFloatType{15} ]

(in python) x[7]=
7
[ CPUFloatType{} ]

(in python) x[True, False]=
[ CPUFloatType{0,15} ]

(in python) x[1::2]=
  1
  3
  5
  7
  9
 11
 13
[ CPUFloatType{7} ]

(in python) x[::2]=
  0
  2
  4
  6
  8
 10
 12
 14
[ CPUFloatType{8} ]



In [3]:
torch::Tensor x = torch::zeros(15);

for (int i = 0; i < 15; i++) {
    x[i] = i;
}
    
std::cout << "x =" << std::endl;
std::cout << (x) << std::endl << std::endl;

/* *
 *  Setter ops
 */
std::cout << "(in python) x[None] = 1" << std::endl;
std::cout << (x.index_put_({torch::indexing::None}, 1)) << std::endl << std::endl;

std::cout << "(in python) x[Ellipsis, ...] = 2" << std::endl;
std::cout << (x.index_put_({torch::indexing::Ellipsis, "..."}, 2)) << std::endl << std::endl;

std::cout << "(in python) x[7] = 3" << std::endl;
std::cout << (x.index_put_({7}, 3)) << std::endl << std::endl;

std::cout << "(in python) x[True, False] = 4" << std::endl;
std::cout << (x.index_put_({true,false,true,false,true,false,true,false,true,false,false,true,false,true,false}, 4)) << std::endl << std::endl;

std::cout << "(in python) x[1::2] = 5" << std::endl;
std::cout << (x.index_put_({torch::indexing::Slice(1, torch::indexing::None, 2)}, 5)) << std::endl << std::endl;

std::cout << "(in python) x[::2] = 6" << std::endl;
std::cout << (x.index_put_({torch::indexing::Slice(torch::indexing::None, torch::indexing::None, 2)}, 6)) << std::endl << std::endl;


x =
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
[ CPUFloatType{15} ]

(in python) x[None] = 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[ CPUFloatType{15} ]

(in python) x[Ellipsis, ...] = 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
[ CPUFloatType{15} ]

(in python) x[7] = 3
 2
 2
 2
 2
 2
 2
 2
 3
 2
 2
 2
 2
 2
 2
 2
[ CPUFloatType{15} ]

(in python) x[True, False] = 4
 2
 2
 2
 2
 2
 2
 2
 3
 2
 2
 2
 2
 2
 2
 2
[ CPUFloatType{15} ]

(in python) x[1::2] = 5
 2
 5
 2
 5
 2
 5
 2
 5
 2
 5
 2
 5
 2
 5
 2
[ CPUFloatType{15} ]

(in python) x[::2] = 6
 6
 5
 6
 5
 6
 5
 6
 5
 6
 5
 6
 5
 6
 5
 6
[ CPUFloatType{15} ]



### Python 与 C++ 索引类型对照表

|Python | C++ (assuming using namespace torch::indexing) |
|:---|:---|
|None|None|
|Ellipsis|Ellipsis|
|...|"..."|
|123|123|
|True|true|
|False|false|
|: or ::|Slice() or Slice(None, None) or Slice(None, None, None)|
|1: or 1::|Slice(1, None) or Slice(1, None, None)|
|:3 or :3:|Slice(None, 3) or Slice(None, 3, None)|
|::2|Slice(None, None, 2)|
|1:3|Slice(1, 3)|
|1::2|Slice(1, None, 2)|
|:3:2|Slice(None, 3, 2)|
|1:3:2|Slice(1, 3, 2)|
|torch.tensor([1, 2])|torch::tensor({1, 2})|

### 其它索引操作

In [34]:
torch::Tensor x = torch::zeros({3,4});

for (int i = 0; i < 3; i++) {
    for (int j = 0; j < 4; j++) {
        x[i][j] = i*4+j;
    }
}
    
std::cout << "x =" << std::endl;
std::cout << (x) << std::endl << std::endl;


/** Warning:
 *  a known issue: with xeus-cling, if you call torch::from_blob,
 *  there will be a link error, 
 *  
 *  IncrementalExecutor::executeFunction: 
 *  symbol '__emutls_v._ZSt11__once_call' unresolved 
 *  while linking function '_GLOBAL__sub_I_cling_module_8'!
 *
 *
 *  so I write this example in pure 
 *  c++ code in folder : 'cpp_project/basic_ops'.
 */
// std::vector<int32_t> v = {1,2,7,8}; 
// auto idx = torch::from_blob(v.data(), v.size(), torch::kInt32);


std::cout << "<<< tensor.index_select >>>" << std::endl;
torch::Tensor idx = torch::arange(1,3);
std::cout << (idx) << std::endl << std::endl;

std::cout << "x.index_select(dim = 0, index = idx) =" << std::endl;
std::cout << (x.index_select(0,idx)) << std::endl << std::endl;

std::cout << "x.index_select(dim = 1, index = idx) =" << std::endl;
std::cout << (x.index_select(1,idx)) << std::endl << std::endl;


std::cout << "<<< tensor.masked_select >>>" << std::endl;
torch::Tensor mask = x > 5;
std::cout << (mask) << std::endl << std::endl;
std::cout << "x.masked_select(mask = mask) =" << std::endl;
std::cout << (x.masked_select(mask)) << std::endl << std::endl;

std::cout << "<<< tensor.nonzero >>>" << std::endl;
std::cout << "注意，返回值是非零元素下标，即x,y "<< std::endl << "x.nonzero() =" << std::endl;
std::cout << (x.nonzero()) << std::endl << std::endl;

std::cout << "<<< tensor.gather >>>" << std::endl;
std::cout << "注意，index tensor必须指明类型为int64" << std::endl;
torch::Tensor g = torch::ones({2,2}, torch::kInt64);
std::cout << (x.gather(0, g)) << std::endl << std::endl;
// std::cout << (torch::gather(x, 0, g, false)) << std::endl << std::endl;


x =
  0   1   2   3
  4   5   6   7
  8   9  10  11
[ CPUFloatType{3,4} ]

<<< tensor.index_select >>>
 1
 2
[ CPULongType{2} ]

x.index_select(dim = 0, index = idx) =
  4   5   6   7
  8   9  10  11
[ CPUFloatType{2,4} ]

x.index_select(dim = 1, index = idx) =
  1   2
  5   6
  9  10
[ CPUFloatType{3,2} ]

<<< tensor.masked_select >>>
 0  0  0  0
 0  0  1  1
 1  1  1  1
[ CPUBoolType{3,4} ]

x.masked_select(mask = mask) =
  6
  7
  8
  9
 10
 11
[ CPUFloatType{6} ]

<<< tensor.nonzero >>>
注意，返回值是非零元素下标，即x,y 
x.nonzero() =
 0  1
 0  2
 0  3
 1  0
 1  1
 1  2
 1  3
 2  0
 2  1
 2  2
 2  3
[ CPULongType{11,2} ]

<<< tensor.gather >>>
注意，index tensor必须指明类型为int64
 4  5
 4  5
[ CPUFloatType{2,2} ]



### 改变形状

In [36]:
/* *
 *  Change shapes
 */
torch::Tensor y = x.view({4, -1});
std::cout << "y =" << std::endl;
std::cout << (y) << std::endl << std::endl;

y =
  0   1   2
  3   4   5
  6   7   8
  9  10  11
[ CPUFloatType{4,3} ]

